Day 09 - Part 1: Finding peaks

In [100]:
import pandas as pd
data = pd.read_csv('test_data.csv', header=None)

def find_sum_peaks(data:pd.DataFrame):
    from findpeaks import findpeaks
    import numpy as np

    numbers = [str(no[0]) for no in data.values.tolist()]
    original_array = []
    for idx in range(len(numbers)):
        original_array.append([int(i) for i in numbers[idx]])

    # 100 is added to transpose vals
    transposed = np.array(original_array) + 100
    for val, rep in zip(range(10), range(9,-1,-1)):
        transposed = np.where(transposed == val + 100, rep, transposed)

    # Find peaks and cby mask method
    fp = findpeaks(method='mask')
    peaks = fp.fit(transposed)
    values = np.array(original_array)[peaks['Xdetect']] + 1
    return sum(values)

find_sum_peaks(data)

[findpeaks] >Finding peaks in 2d-array using mask method..
[findpeaks] >Scaling image between [0-255] and to uint8
[findpeaks] >Conversion to gray image.
[findpeaks] >WARNING: Conversion to gray not possible.
[findpeaks] >Denoising with [fastnl], window: [3].
[findpeaks] >Detect peaks using the mask method with limit=0.
[findpeaks] >Fin.


15

In [99]:
data = pd.read_csv('input_data.csv', header=None)
find_sum_peaks(data)

[findpeaks] >Finding peaks in 2d-array using mask method..
[findpeaks] >Scaling image between [0-255] and to uint8
[findpeaks] >Conversion to gray image.
[findpeaks] >WARNING: Conversion to gray not possible.
[findpeaks] >Denoising with [fastnl], window: [3].
[findpeaks] >Detect peaks using the mask method with limit=0.
[findpeaks] >Fin.


468

Part 2: Find largest basin

In [207]:
data = pd.read_csv('input_data.csv', header=None)

def find_prod_basin(data):

    import pandas as pd
    import numpy as np
    import cv2 as cv


    numbers = [str(no[0]) for no in data.values.tolist()]
    original_array = []
    for idx in range(len(numbers)):
        original_array.append([int(i) for i in numbers[idx]])
    original_array = np.array(original_array)

    # Turn image to binary black and while. As 9's are white and non-9's are 0
    img = np.where(original_array != 9,255, original_array)
    img = np.where(img == 9, 0, img)
    img = img.astype('uint8')

    # Add padding to image
    img = cv.copyMakeBorder(img, 2, 2, 2, 2, cv.BORDER_CONSTANT, None, 0)

    # Crete distance transformation map to determine which points has largest distance from borders which also means largest in this example
    dist_transform = cv.distanceTransform(img, cv.DIST_L2, 0)

    # Find index of top 20 values
    top_N = 1000
    idx = np.argpartition(dist_transform, dist_transform.size - top_N, axis=None)[-top_N:]
    results= np.column_stack(np.unravel_index(idx, dist_transform.shape))
    tups = [tuple(result) for result in results]

    # Find fill size
    h, w = img.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    flood_fill = [cv.floodFill(img, mask, [tup[1],tup[0]], 255)[0] for tup in tups]

    # Find top 3 largest areas and get product
    top_areas = np.argpartition(flood_fill, -3)[-3:]
    top_areas = np.array([flood_fill[idx] for idx in top_areas])
    product = top_areas.prod()
    return product

find_prod_basin(data)

1280496